In [ ]:
##from a previous idea i had which was braindead do not use 

def remove_encoding_tokens(encoding_list):
    for element in encoding_list:

        if element in grammer_token_list:

            encoding_list = encoding_list[encoding_list!=element]
    return encoding_list

In [ ]:
##can probably completely delete i have no idea what this does anyomore 
model_name = "bert-base-uncased"
max_length = 1000
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)
train_encodings = tokenizer(X, truncation=True, padding=True, max_length=max_length,is_split_into_words=False)
train_dataset = CardDataset(train_encodings, y)
nlp = TFBertModel.from_pretrained('bert-base-uncased')

In [ ]:
# stole this from womewheer for some reason not sure why 

class CardDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
grammer_things = '.[]{)(}!@#$%^&*`|;:,<>?- "/ '
grammer_token_list = np.array(tokenizer.encode(grammer_things.split(),add_special_tokens=False,is_split_into_words=True))[None,:][0]
print(grammer_token_list)

In [ ]:
# this shit is probably completely irrelevant I'm pretty sure 

model=BertForSequenceClassification.from_pretrained(model_name, num_labels=1)
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments( #TODO fuck with these 
    num_train_epochs=1,
    per_device_train_batch_size=16,  # batch size per device during training
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    logging_steps=100,
    evaluation_strategy="steps",
    output_dir='.'
)

trainer = Trainer( model=model,args=training_args,train_dataset=train_dataset)
trainer.train()